In [3]:
import re
import string
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.corpus import stopwords

import random
from typing import List, Optional

In [4]:
# Zorg dat stopwords beschikbaar zijn
try:
    DUTCH_STOPWORDS = set(stopwords.words("dutch"))
except LookupError:
    nltk.download("stopwords")
    DUTCH_STOPWORDS = set(stopwords.words("dutch"))

# Optioneel: extra domein-specifieke "ruiswoorden"
EXTRA_NOISE = {
    "bij", "voor", "met", "door", "zonder", "over",
    "doelgroep", "werk", "werken", "proces", "praktijk",
    "ontwikkeling", "ontwikkelen", "gaan",
    "leren", "school", "module", "modules", "thema",
    "student", "opleiding", "kun" , "vanuit",
    "eigen", "zelf", "samen", "samenwerken",
    "jaar", "week", "periode", "naasten", "daarnaast",
    "minor", "studenten", "programma", "keuzemodule",
    "casus", "casussen", "cases", "vraagstukken",
    "stage", "stageschool", "kennismakingsstage",
    "kennis", "vaardigheid", "vaardigheden", "ervaring", "ervaringen",
    "lessen", "onderwerpen", "theorie", "praktijk", "praktische", "inhoudelijke",
    "mee", "doe", "vinden", "vind", "kies", "openstaan",
    "belangrijk", "positief", "mogelijkheden","mogelijkheid",
    "impact", "betekenis", "betekent", "betekenen",
    "you", "your", "are", "will", "what", "then", "like", "choose",
    "interested", "experiencing", "hbo", "and", "the", "persoonlijke",
    "learning", "denken", "maken", "business", "verdieping", "emgeving",
    "bouwen", "thinking", "branding", "maken", "urban", "veiligheid",
    "nieuwe", "test", "gebouwde", "concept", "project", "omgeving", "actuele", "acute",
    "yellow", "belt", "serious", "hrm", "mensen", "snel", "binnen", "materialen", 
    "active", "druk", "context", "leven", "complexe" , "brede", "for", "jouw", "manieren"
}

TEXT_STOPWORDS = DUTCH_STOPWORDS | EXTRA_NOISE

PUNCT_TABLE = str.maketrans("", "", string.punctuation + "’‘“”´`")

In [5]:
def clean_text_for_matching(text: str) -> str:
    """
    Maakt tekst klaar voor matching:
    - lowercase
    - verwijder punctuation
    - verwijder cijfers
    - verwijder NL stopwoorden (+ extra noise)
    """
    if not isinstance(text, str):
        return ""
    
    # lowercasing
    text = text.lower()
    
    # punctuation verwijderen
    text = text.translate(PUNCT_TABLE)
    
    # cijfers eruit
    text = re.sub(r"\d+", " ", text)
    
    # meerdere spaties
    text = re.sub(r"\s+", " ", text).strip()
    
    # stopwoorden filteren
    tokens = [
        tok for tok in text.split()
        if tok not in TEXT_STOPWORDS and len(tok) > 2
    ]
    
    return " ".join(tokens)


In [6]:
# Voorbeeld: pas dit pad aan naar jouw echte bron
df = pd.read_csv("Uitgebreide_VKM_dataset_cleaned2.csv")  # of je eigen load-functie

# Verwachte kolommen (minimaal):
# id, name, shortdescription, location, studycredit, level, module_tags, ...
# We maken/overschrijven een 'clean_text'-kolom op basis van relevante velden.

TEXT_COLUMNS = ["name", "shortdescription"]  # breid uit als je meer wilt meenemen

def build_raw_text(row: pd.Series) -> str:
    parts = []
    for col in TEXT_COLUMNS:
        if col in row and isinstance(row[col], str):
            parts.append(row[col])
    return " ".join(parts)

df["raw_text"] = df.apply(build_raw_text, axis=1)
df["clean_text"] = df["raw_text"].apply(clean_text_for_matching)



In [7]:
# Vectorizer eenmalig fitten op de complete dataset
vectorizer = TfidfVectorizer(
        ngram_range=(1, 2),
        max_df=0.8,      # woorden die in >80% van de modules zitten eruit
        min_df=2,        # woorden die maar 1x voorkomen eruit
)
X = vectorizer.fit_transform(df["clean_text"])

FEATURE_NAMES = vectorizer.get_feature_names_out()
X.shape, len(FEATURE_NAMES)


((207, 455), 455)

In [8]:
def _format_term_list(terms: List[str]) -> str:
    """
    Maak een nette NL opsomming:
    - 'A'
    - 'A' en 'B'
    - 'A', 'B' en 'C'
    """
    if not terms:
        return ""
    if len(terms) == 1:
        return f"'{terms[0]}'"
    if len(terms) == 2:
        return f"'{terms[0]}' en '{terms[1]}'"
    # 3 of meer
    quoted = [f"'{t}'" for t in terms]
    hoofd = ", ".join(quoted[:-1])
    laatste = quoted[-1]
    return f"{hoofd} en {laatste}"


In [9]:
def build_reason(match_terms: List[str], module_name: Optional[str] = None, score: Optional[float] = None) -> str:
    """
    Genereer een korte NL tekst waarom de module past.
    - match_terms: termen waarop je matcht (interesses / keywords)
    - module_name: optioneel, naam van de module voor iets persoonlijkere tekst
    - score: optioneel, genormaliseerde similarity-score (0–1)
    """
    terms_str = _format_term_list(match_terms)

    # score -> indicatie hoe sterk de match is
    if score is None:
        kwalificatie = "goed"
    elif score >= 0.8:
        kwalificatie = "erg goed"
    elif score >= 0.6:
        kwalificatie = "goed"
    else:
        kwalificatie = "redelijk"

    # Geen specifieke termen: algemene uitleg
    if not match_terms:
        templates = [
            "Deze module sluit {kwalificatie} aan bij je interesses op basis van tekstuele overeenkomsten.",
            "Op basis van de overeenkomst tussen jouw profiel en de modulebeschrijving lijkt deze module {kwalificatie} bij je te passen.",
            "Deze module lijkt inhoudelijk {kwalificatie} aan te sluiten bij wat je interessant vindt."
        ]
    else:
        # Met match_terms
        if module_name:
            templates = [
                "Je interesse in {terms} komt duidelijk terug in '{module}', waardoor deze module {kwalificatie} bij je aansluit.",
                "Omdat {terms} centraal staan in '{module}', past deze module {kwalificatie} bij jouw interesses.",
                "In '{module}' komen {terms} aan bod, wat goed aansluit bij jouw interesses."
            ]
        else:
            templates = [
                "Deze module sluit {kwalificatie} aan bij je interesses in {terms}.",
                "Omdat {terms} in deze module aan bod komen, lijkt deze {kwalificatie} bij je te passen.",
                "Je interesse in {terms} komt terug in de inhoud van deze module, waardoor deze goed bij je past."
            ]

    template = random.choice(templates)
    return template.format(
        kwalificatie=kwalificatie,
        terms=terms_str,
        module=module_name if module_name else ""
    )


In [10]:
def extract_match_terms(student_vec, module_vec, feature_names, max_terms: int = 8) -> List[str]:
    """
    Bepaal welke termen (woorden) zowel in de studentvector
    als in de modulevector voorkomen.
    """
    # indices waar de vector niet 0 is
    student_idx = set(student_vec.nonzero()[1])
    module_idx = set(module_vec.nonzero()[1])

    shared_idx = sorted(student_idx & module_idx)
    terms = [feature_names[i] for i in shared_idx]

    return terms[:max_terms]


In [11]:
def recommend_modules(student_profile: str, top_n: int = 5, studycredit: Optional[int] = None, level: Optional[str] = None, location: Optional[str] = None) -> pd.DataFrame:
    """
    Geeft top N modules voor een studentprofiel,
    inclusief:
    - similarity (genormaliseerd 0–1 binnen de selectie)
    - similarity_raw (werkelijke cosine score)
    - match_terms
    - reason (NL-uitleg)
    """

    # Start met volledige df
    filtered_df = df.copy()

    # Filter op studiepunten, niveau, locatie
    if studycredit is not None:
        filtered_df = filtered_df[filtered_df["studycredit"] == studycredit]

    if level is not None:
        filtered_df = filtered_df[filtered_df["level"].isin([level])]

    if location is not None:
        filtered_df = filtered_df[
            filtered_df["location"].str.contains(location, case=False, na=False)
        ]

    # Niks over? Return lege tabel met juiste kolommen
    if filtered_df.empty:
        return pd.DataFrame(
            columns=[
                "id", "name", "shortdescription", "similarity",
                "similarity_raw", "location", "studycredit",
                "level", "module_tags", "match_terms", "reason"
            ]
        )

    # Studentprofiel vectoriseren
    clean_profile = clean_text_for_matching(student_profile)
    student_vec = vectorizer.transform([clean_profile])

    # Bijbehorende rijen uit X pakken
    row_indices = filtered_df.index.to_numpy()
    X_filtered = X[row_indices]

    # Cosine similarity
    sims = cosine_similarity(student_vec, X_filtered).flatten()

    # In kopie wegschrijven
    filtered_df = filtered_df.copy()
    filtered_df["similarity_raw"] = sims

    # Normaliseren naar 0–1 binnen deze selectie
    max_sim = sims.max()
    if max_sim > 0:
        norm_sims = sims / max_sim
    else:
        norm_sims = sims  # alles 0

    filtered_df["similarity"] = norm_sims

    # Sorteren en top N pakken
    top = filtered_df.sort_values("similarity", ascending=False).head(top_n)

    match_terms_list: List[List[str]] = []
    reasons: List[str] = []

    for idx in top.index:
        # positie van deze rij binnen X_filtered
        pos = np.where(row_indices == idx)[0][0]
        module_vec = X_filtered[pos]

        terms = extract_match_terms(student_vec, module_vec, FEATURE_NAMES)
        match_terms_list.append(terms)

        reasons.append(
            build_reason(
                terms,
                module_name=top.at[idx, "name"],
                score=top.at[idx, "similarity"]  # genormaliseerde score
            )
        )

    # Kolommen selecteren en verrijken
    top = top[["id", "name", "shortdescription", "similarity", "similarity_raw", "location", "studycredit", "level", "module_tags"]].copy()
    top["match_terms"] = match_terms_list
    top["reason"] = reasons

    return top.reset_index(drop=True)


In [ ]:
sam_profiel = """
Ik heb een achtergrond in de zorg en ik ben geïnteresseerd in hoe technologie kan helpen om zorg en welzijn slimmer te organiseren. 
Dingen als smart health, e-health, zorg op afstand en innovatieve oplossingen voor ouderen spreken me erg aan. 
Ik werk graag met praktische toepassingen van technologie in de zorgpraktijk en vind het leuk om in teams aan echte cases te werken.
"""
pd.set_option("display.max_colwidth", None)

result = recommend_modules(
    student_profile=sam_profiel,
    top_n=5,
    # studycredit=15,      # of None
    # level="NLQF5",       # of None
    # location="Breda"     # of None
)

result


,id,name,shortdescription,similarity,similarity_raw,location,studycredit,level,module_tags,match_terms,reason
0,176,Technologie die ècht werkt: innovatie in zorg en welzijn,"Technologie, smart health, robotica, zorg, welzijn",1.000000,0.873147,Breda,15,NLQF5,"['technologie', 'smart', 'health', 'robotica', 'zorg', 'welzijn']","[health, smart, smart health, technologie, welzijn, zorg, zorg welzijn]","Omdat 'health', 'smart', 'smart health', 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' centraal staan in 'Technologie die ècht werkt: innovatie in zorg en welzijn', past deze module erg goed bij jouw interesses."
1,171,Technologie in zorg en welzijn,"Technologie, zorg, welzijn",0.877949,0.766579,Den Bosch,30,NLQF6,"['technologie', 'zorg', 'welzijn']","[technologie, welzijn, zorg, zorg welzijn]","In 'Technologie in zorg en welzijn' komen 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' aan bod, wat goed aansluit bij jouw interesses."
2,172,Zorg dichtbij,"Zorg op afstand, technologie, implementeren, zorg, veranderkunde",0.799766,0.698313,Breda,15,NLQF5,"['zorg', 'afstand', 'technologie', 'implementeren', 'veranderkunde']","[technologie, zorg]","Je interesse in 'technologie' en 'zorg' komt duidelijk terug in 'Zorg dichtbij', waardoor deze module goed bij je aansluit."
3,173,Langer thuis in de wijk,"wijkgericht werken, eigen regie, zorg, welzijn, ouderen",0.672773,0.587430,Breda,15,NLQF5,"['wijkgericht', 'regie', 'zorg', 'welzijn', 'ouderen']","[welzijn, zorg, zorg welzijn]","Je interesse in 'welzijn', 'zorg' en 'zorg welzijn' komt duidelijk terug in 'Langer thuis in de wijk', waardoor deze module goed bij je aansluit."
4,199,Acuut Complexe Zorg,"Zorg, welzijn, acute zorg",0.600137,0.524008,Breda,30,NLQF6,"['zorg', 'welzijn']","[welzijn, zorg, zorg welzijn]","In 'Acuut Complexe Zorg' komen 'welzijn', 'zorg' en 'zorg welzijn' aan bod, wat goed aansluit bij jouw interesses."


In [13]:
gerda_profiel = """
Ik ben vooral geïnteresseerd in data, kunstmatige intelligentie en slimme systemen. 
Ik vind onderwerpen zoals machine learning, big data, Internet of Things, smart industry en automatisering supertof. 
Ik werk graag analytisch, met datasets en tools, en ik vind het leuk om oplossingen te bouwen die processen slimmer maken.
"""
pd.set_option("display.max_colwidth", None)

result = recommend_modules(
    student_profile=gerda_profiel,
    top_n=5,
    # studycredit=15,      # of None
    # level="NLQF5",       # of None
    # location="Breda"     # of None
)

result

,id,name,shortdescription,similarity,similarity_raw,location,studycredit,level,module_tags,match_terms,reason
0,333,Smart Industry & Internet of Things,"big data, smart robotics, Artificial Intelligence, machine learning, IoT, smart industry, Industry 4.0",1.000000,0.609941,Breda,30,NLQF6,"['big', 'data', 'smart', 'robotics', 'artificial', 'intelligence', 'machine', 'iot', 'industry', '40']","[big, big data, data, industry, machine, smart, smart industry]","In 'Smart Industry & Internet of Things' komen 'big', 'big data', 'data', 'industry', 'machine', 'smart' en 'smart industry' aan bod, wat goed aansluit bij jouw interesses."
1,255,Module 2.3 Data gestuurd Ontwerp (Data Driven Design),"Data driven design, big data, BIM/GIS, parametrisch ontwerpen, virtual reality, storytelling",0.640262,0.390522,Den Bosch en Tilburg,15,NLQF5,"['data', 'driven', 'design', 'big', 'bimgis', 'parametrisch', 'ontwerpen', 'virtual', 'reality', 'storytelling']","[big, big data, data]","Je interesse in 'big', 'big data' en 'data' komt duidelijk terug in 'Module 2.3 Data gestuurd Ontwerp (Data Driven Design)', waardoor deze module goed bij je aansluit."
2,355,Chemical modelling,"Big data, sensoriek, programmeren, modellering, AI",0.633281,0.386264,Den Bosch,30,NLQF6,"['big', 'data', 'sensoriek', 'programmeren', 'modellering', 'ai']","[big, big data, data]","In 'Chemical modelling' komen 'big', 'big data' en 'data' aan bod, wat goed aansluit bij jouw interesses."
3,323,Advanced computer vision,Smart Industry,0.602747,0.367640,Breda,15,NLQF6,"['smart', 'industry']","[industry, smart, smart industry]","In 'Advanced computer vision' komen 'industry', 'smart' en 'smart industry' aan bod, wat goed aansluit bij jouw interesses."
4,319,Green Belt,Smart Industry / Smart Health,0.545765,0.332884,Tilburg,15,NLQF5,"['smart', 'industry', 'health']","[industry, smart, smart industry]","Omdat 'industry', 'smart' en 'smart industry' centraal staan in 'Green Belt', past deze module redelijk bij jouw interesses."


In [14]:
klaas_profiel = """
Ik vind innovatie en vernieuwing binnen organisaties heel interessant. 
Ik werk graag aan nieuwe ideeën, producten en diensten en denk na over hoe je die in de praktijk kunt implementeren. 
Thema’s zoals innovatiemanagement, verandermanagement, business modellen en het verbeteren van processen spreken me erg aan. 
Ik ben minder bezig met diep technische details, maar meer met strategie en organisatie.

"""
pd.set_option("display.max_colwidth", None)

result = recommend_modules(
    student_profile=klaas_profiel,
    top_n=5,
    # studycredit=15,      # of None
    # level="NLQF5",       # of None
    # location="Breda"     # of None
)

result

,id,name,shortdescription,similarity,similarity_raw,location,studycredit,level,module_tags,match_terms,reason
0,331,Innovatiemanagement,"Het centrale thema van deze minor heeft betrekking op technologie en innovatie van nieuwe producten/diensten en processen binnen bestaande en nieuwe organisatie en de mogelijkheden van innovatie om verbeteringen in de bedrijfsvoering van organisaties door te voeren. Innovatie of vernieuwing is het invoeren van nieuwe ideeën, goederen, diensten en processen. Innovatie kan plaatsvinden binnen organisaties maar ook binnen bredere verbanden. Het proces van innovatie draait om dingen op een nieuwe (en zo mogelijk ook betere) manier aan te pakken. Het centrale thema van deze minor heeft betrekking op technologie en innovatie van nieuwe producten/diensten en processen binnen bestaande en nieuwe organisatie en de mogelijkheden van innovatie om verbeteringen in de bedrijfsvoering van organisaties door te voeren. Innovatie of vernieuwing is het invoeren van nieuwe ideeën, goederen, diensten en processen. Innovatie kan plaatsvinden binnen organisaties maar ook binnen bredere verbanden. Het proces van innovatie draait om dingen op een nieuwe (en zo mogelijk ook betere) manier aan te pakken.",1.000000,0.529271,Tilburg,30,NLQF6,"['innovatie draait', 'innovatie plaatsvinden', 'innovatie verbeteringen', 'innovatie', 'processen innovatie', 'innovatie vernieuwing', 'voeren innovatie', 'vernieuwing invoeren']","[innovatie, organisatie, organisaties, processen]","In 'Innovatiemanagement' komen 'innovatie', 'organisatie', 'organisaties' en 'processen' aan bod, wat goed aansluit bij jouw interesses."
1,311,Continu Verbeteren,"Verbeteropdracht, Greenbelt, bedrijfsleven, six sigma",0.661438,0.350080,Den Bosch,30,NLQF6,"['verbeteropdracht', 'greenbelt', 'bedrijfsleven', 'six', 'sigma']",[verbeteren],"Je interesse in 'verbeteren' komt duidelijk terug in 'Continu Verbeteren', waardoor deze module goed bij je aansluit."
2,272,Duurzame Business Modellen,"Duurzaamheid, business modellen, advisering",0.524213,0.277451,Den Bosch,15,NLQF5,"['duurzaamheid', 'modellen', 'advisering']",[modellen],"Je interesse in 'modellen' komt duidelijk terug in 'Duurzame Business Modellen', waardoor deze module redelijk bij je aansluit."
3,327,Analyseren van software en infrastructuur (1.3 BIM),"In deze module leer je de software en infrastructuur inrichting van een organisatie kennen adviseer je een organisatie bij de aanschaf van een nieuw (software) pakket alsmede de impact die dit heeft op de bestaande organisatie (processen, infrastructuur en andere software). In deze module leer je de software en infrastructuur inrichting van een organisatie kennen adviseer je een organisatie bij de aanschaf van een nieuw (software) pakket alsmede de impact die dit heeft op de bestaande organisatie (processen, infrastructuur en andere software).",0.463791,0.245471,Breda,15,NLQF5,"['software infrastructuur', 'infrastructuur software', 'software', 'leer software', 'software pakket', 'adviseer organisatie', 'processen infrastructuur', 'nieuw software', 'inrichting organisatie']","[organisatie, processen]","Omdat 'organisatie' en 'processen' centraal staan in 'Analyseren van software en infrastructuur (1.3 BIM)', past deze module redelijk bij jouw interesses."
4,235,BI Innovatie,"innovatie, innovatie methoden, casework, business models, responsible",0.412929,0.218551,Den Bosch,15,NLQF5,"['innovatie', 'methoden', 'casework', 'models', 'responsible']",[innovatie],"Omdat 'innovatie' centraal staan in 'BI Innovatie', past deze module redelijk bij jouw interesses."


In [15]:
mathilda_profiel = """
Ik vind het leuk om creatieve oplossingen te ontwerpen met behulp van technologie. 
Onderwerpen zoals data driven design, UX/UI, virtual reality, storytelling en het visueel maken van informatie spreken me erg aan. 
Ik wil graag leren hoe je technieken zoals data-analyse en visualisatie inzet om betere ontwerpen en gebruikservaringen te maken.
"""
pd.set_option("display.max_colwidth", None)

result = recommend_modules(
    student_profile=mathilda_profiel,
    top_n=5,
    # studycredit=15,      # of None
    # level="NLQF5",       # of None
    # location="Breda"     # of None
)

result

,id,name,shortdescription,similarity,similarity_raw,location,studycredit,level,module_tags,match_terms,reason
0,255,Module 2.3 Data gestuurd Ontwerp (Data Driven Design),"Data driven design, big data, BIM/GIS, parametrisch ontwerpen, virtual reality, storytelling",1.000000,0.562576,Den Bosch en Tilburg,15,NLQF5,"['data', 'driven', 'design', 'big', 'bimgis', 'parametrisch', 'ontwerpen', 'virtual', 'reality', 'storytelling']","[data, design, driven, ontwerpen, reality, storytelling, virtual, virtual reality]","In 'Module 2.3 Data gestuurd Ontwerp (Data Driven Design)' komen 'data', 'design', 'driven', 'ontwerpen', 'reality', 'storytelling', 'virtual' en 'virtual reality' aan bod, wat goed aansluit bij jouw interesses."
1,254,Module 2.1/2.3 Toekomstgericht Ontwerpen,"Natuurinclusief Ontwerpen, post humaan denken, transitiedenken, paradigma shift",0.737974,0.415167,Den Bosch en Tilburg,15,NLQF5,"['natuurinclusief', 'ontwerpen', 'post', 'humaan', 'transitiedenken', 'paradigma', 'shift']",[ontwerpen],"Omdat 'ontwerpen' centraal staan in 'Module 2.1/2.3 Toekomstgericht Ontwerpen', past deze module goed bij jouw interesses."
2,321,Virtual en Augmented Reality,"Virtual Reality, Augmented Reality, Digital Twin, Serious Game, User Experience",0.620171,0.348893,Breda,15,NLQF5,"['virtual', 'reality', 'augmented', 'digital', 'twin', 'game', 'user', 'experience']","[reality, virtual, virtual reality]","Omdat 'reality', 'virtual' en 'virtual reality' centraal staan in 'Virtual en Augmented Reality', past deze module goed bij jouw interesses."
3,367,Proefdierkunde,Om in het lab met proefdieren te mogen werken moet je als biomedisch onderzoeker voldoen aan de beroepsvereisten voor biotechnicus zoals bedoeld in artikel 6 lid 1b van de Dierproevenregeling 2014 (voorheen artikel 12 bevoegdheid volgens de Wet op Dierproeven). Om in het lab met proefdieren te mogen werken moet je als biomedisch onderzoeker voldoen aan de beroepsvereisten voor biotechnicus zoals bedoeld in artikel 6 lid 1b van de Dierproevenregeling 2014 (voorheen artikel 12 bevoegdheid volgens de Wet op Dierproeven).,0.536970,0.302086,Breda,15,NLQF5,"['dierproeven lab', 'lab proefdieren', 'beroepsvereisten biotechnicus', 'dierproevenregeling', 'dierproevenregeling 2014', 'lid dierproevenregeling', 'biotechnicus zoals', 'biomedisch', 'biotechnicus', 'dierproeven']",[zoals],"Je interesse in 'zoals' komt duidelijk terug in 'Proefdierkunde', waardoor deze module redelijk bij je aansluit."
4,370,Sonic Narratives,"Sound design, opname, productie, postproductie, storytelling",0.504992,0.284096,Breda,15,NLQF5,"['sound', 'design', 'opname', 'productie', 'postproductie', 'storytelling']","[design, storytelling]","Omdat 'design' en 'storytelling' centraal staan in 'Sonic Narratives', past deze module redelijk bij jouw interesses."


In [16]:
from typing import List, Optional
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def recommend_modules_with(
    df: pd.DataFrame,
    vectorizer,
    X,
    feature_names,
    student_profile: str,
    top_n: int = 5,
    studycredit: Optional[int] = None,
    level: Optional[str] = None,
    location: Optional[str] = None
) -> pd.DataFrame:
    """
    Zelfde idee als recommend_modules, maar gebruikt de meegegeven
    vectorizer, X en feature_names. Handig voor experimenten met TF-IDF.
    """
    filtered_df = df.copy()

    # Filter op studiepunten, niveau, locatie
    if studycredit is not None:
        filtered_df = filtered_df[filtered_df["studycredit"] == studycredit]

    if level is not None:
        filtered_df = filtered_df[filtered_df["level"].isin([level])]

    if location is not None:
        filtered_df = filtered_df[
            filtered_df["location"].str.contains(location, case=False, na=False)
        ]

    # Niks over? -> lege tabel terug
    if filtered_df.empty:
        return pd.DataFrame(
            columns=[
                "id", "name", "shortdescription", "similarity",
                "similarity_raw", "location", "studycredit",
                "level", "module_tags", "match_terms", "reason"
            ]
        )

    # Studentprofiel vectoriseren
    clean_profile = clean_text_for_matching(student_profile)
    student_vec = vectorizer.transform([clean_profile])

    # Bijbehorende rijen uit X pakken
    row_indices = filtered_df.index.to_numpy()
    X_filtered = X[row_indices]

    # Cosine similarity
    sims = cosine_similarity(student_vec, X_filtered).flatten()

    filtered_df = filtered_df.copy()
    filtered_df["similarity_raw"] = sims

    # Normaliseren naar 0–1 binnen deze selectie
    max_sim = sims.max()
    if max_sim > 0:
        norm_sims = sims / max_sim
    else:
        norm_sims = sims

    filtered_df["similarity"] = norm_sims

    # Sorteren en top N pakken
    top = filtered_df.sort_values("similarity", ascending=False).head(top_n)

    match_terms_list: List[List[str]] = []
    reasons: List[str] = []

    for idx in top.index:
        # positie van deze rij binnen X_filtered
        pos = np.where(row_indices == idx)[0][0]
        module_vec = X_filtered[pos]

        terms = extract_match_terms(student_vec, module_vec, feature_names)
        match_terms_list.append(terms)

        reasons.append(
            build_reason(
                match_terms=terms,
                module_name=top.at[idx, "name"],
                score=top.at[idx, "similarity"]
            )
        )

    top = top[[
        "id",
        "name",
        "shortdescription",
        "similarity",
        "similarity_raw",
        "location",
        "studycredit",
        "level",
        "module_tags",
    ]].copy()

    top["match_terms"] = match_terms_list
    top["reason"] = reasons

    return top.reset_index(drop=True)

In [17]:
tfidf_configs = {
    "baseline": TfidfVectorizer(),
    "ngrams_1_2": TfidfVectorizer(ngram_range=(1, 2)),
    "pruned_vocab": TfidfVectorizer(
        ngram_range=(1, 2),
        max_df=0.8,      # woorden die in >80% van de modules zitten eruit
        min_df=2,        # woorden die maar 1x voorkomen eruit
    ),
    "sublinear": TfidfVectorizer(
        ngram_range=(1, 2),
        sublinear_tf=True
    ),
}

In [18]:
student_profile = """
Ik heb een achtergrond in de zorg en ik ben geïnteresseerd in hoe technologie kan helpen om zorg en welzijn slimmer te organiseren. 
Dingen als smart health, e-health, zorg op afstand en innovatieve oplossingen voor ouderen spreken me erg aan. 
Ik werk graag met praktische toepassingen van technologie in de zorgpraktijk en vind het leuk om in teams aan echte cases te werken.
"""

for name, vec in tfidf_configs.items():
    print("=== Config:", name, "===\n")

    # Vectorizer fitten op de clean_text van alle modules
    X_exp = vec.fit_transform(df["clean_text"])
    feature_names = vec.get_feature_names_out()

    # Aanbevelingen genereren met deze vectorizer
    result = recommend_modules_with(
        df=df,
        vectorizer=vec,
        X=X_exp,
        feature_names=feature_names,
        student_profile=student_profile,
        top_n=5,
        # studycredit=15,       # pas aan of zet op None
        # level="NLQF5",        # pas aan of zet op None
        # location="Breda"      # pas aan of zet op None
    )

    display(result[["name", "similarity", "similarity_raw", "match_terms", "reason"]])
    print("\n\n")

=== Config: baseline ===



,name,similarity,similarity_raw,match_terms,reason
0,Technologie in zorg en welzijn,1.000000,0.719090,"[technologie, welzijn, zorg]","Omdat 'technologie', 'welzijn' en 'zorg' centraal staan in 'Technologie in zorg en welzijn', past deze module erg goed bij jouw interesses."
1,Technologie die ècht werkt: innovatie in zorg en welzijn,0.889463,0.639604,"[health, smart, technologie, welzijn, zorg]","Je interesse in 'health', 'smart', 'technologie', 'welzijn' en 'zorg' komt duidelijk terug in 'Technologie die ècht werkt: innovatie in zorg en welzijn', waardoor deze module erg goed bij je aansluit."
2,Zorg dichtbij,0.836704,0.601665,"[afstand, technologie, zorg]","In 'Zorg dichtbij' komen 'afstand', 'technologie' en 'zorg' aan bod, wat goed aansluit bij jouw interesses."
3,Acuut Complexe Zorg,0.700425,0.503669,"[welzijn, zorg]","Omdat 'welzijn' en 'zorg' centraal staan in 'Acuut Complexe Zorg', past deze module goed bij jouw interesses."
4,Smart Health,0.462715,0.332733,"[health, smart]","Je interesse in 'health' en 'smart' komt duidelijk terug in 'Smart Health', waardoor deze module redelijk bij je aansluit."





=== Config: ngrams_1_2 ===



,name,similarity,similarity_raw,match_terms,reason
0,Technologie in zorg en welzijn,1.000000,0.513428,"[technologie, welzijn, zorg, zorg welzijn]","In 'Technologie in zorg en welzijn' komen 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' aan bod, wat goed aansluit bij jouw interesses."
1,Technologie die ècht werkt: innovatie in zorg en welzijn,0.990917,0.508765,"[health, smart, smart health, technologie, welzijn, zorg, zorg welzijn]","In 'Technologie die ècht werkt: innovatie in zorg en welzijn' komen 'health', 'smart', 'smart health', 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' aan bod, wat goed aansluit bij jouw interesses."
2,Zorg dichtbij,0.907509,0.465941,"[afstand, technologie, zorg, zorg afstand]","Omdat 'afstand', 'technologie', 'zorg' en 'zorg afstand' centraal staan in 'Zorg dichtbij', past deze module erg goed bij jouw interesses."
3,Acuut Complexe Zorg,0.773606,0.397191,"[welzijn, zorg, zorg welzijn]","Omdat 'welzijn', 'zorg' en 'zorg welzijn' centraal staan in 'Acuut Complexe Zorg', past deze module goed bij jouw interesses."
4,Smart Health,0.699065,0.358920,"[health, smart, smart health]","In 'Smart Health' komen 'health', 'smart' en 'smart health' aan bod, wat goed aansluit bij jouw interesses."





=== Config: pruned_vocab ===



,name,similarity,similarity_raw,match_terms,reason
0,Technologie die ècht werkt: innovatie in zorg en welzijn,1.000000,0.873147,"[health, smart, smart health, technologie, welzijn, zorg, zorg welzijn]","In 'Technologie die ècht werkt: innovatie in zorg en welzijn' komen 'health', 'smart', 'smart health', 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' aan bod, wat goed aansluit bij jouw interesses."
1,Technologie in zorg en welzijn,0.877949,0.766579,"[technologie, welzijn, zorg, zorg welzijn]","Omdat 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' centraal staan in 'Technologie in zorg en welzijn', past deze module erg goed bij jouw interesses."
2,Zorg dichtbij,0.799766,0.698313,"[technologie, zorg]","Je interesse in 'technologie' en 'zorg' komt duidelijk terug in 'Zorg dichtbij', waardoor deze module goed bij je aansluit."
3,Langer thuis in de wijk,0.672773,0.587430,"[welzijn, zorg, zorg welzijn]","Omdat 'welzijn', 'zorg' en 'zorg welzijn' centraal staan in 'Langer thuis in de wijk', past deze module goed bij jouw interesses."
4,Acuut Complexe Zorg,0.600137,0.524008,"[welzijn, zorg, zorg welzijn]","Je interesse in 'welzijn', 'zorg' en 'zorg welzijn' komt duidelijk terug in 'Acuut Complexe Zorg', waardoor deze module goed bij je aansluit."





=== Config: sublinear ===



,name,similarity,similarity_raw,match_terms,reason
0,Technologie in zorg en welzijn,1.000000,0.472889,"[technologie, welzijn, zorg, zorg welzijn]","In 'Technologie in zorg en welzijn' komen 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' aan bod, wat goed aansluit bij jouw interesses."
1,Technologie die ècht werkt: innovatie in zorg en welzijn,0.975648,0.461373,"[health, smart, smart health, technologie, welzijn, zorg, zorg welzijn]","Omdat 'health', 'smart', 'smart health', 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' centraal staan in 'Technologie die ècht werkt: innovatie in zorg en welzijn', past deze module erg goed bij jouw interesses."
2,Zorg dichtbij,0.840290,0.397364,"[afstand, technologie, zorg, zorg afstand]","In 'Zorg dichtbij' komen 'afstand', 'technologie', 'zorg' en 'zorg afstand' aan bod, wat goed aansluit bij jouw interesses."
3,Smart Health,0.820923,0.388205,"[health, smart, smart health]","In 'Smart Health' komen 'health', 'smart' en 'smart health' aan bod, wat goed aansluit bij jouw interesses."
4,Acuut Complexe Zorg,0.633078,0.299376,"[welzijn, zorg, zorg welzijn]","In 'Acuut Complexe Zorg' komen 'welzijn', 'zorg' en 'zorg welzijn' aan bod, wat goed aansluit bij jouw interesses."


In [19]:
def precision_at_k(recommended, relevant, k=5):
    recommended_top_k = recommended[:k]
    hits = sum([1 for item in recommended_top_k if item in relevant])
    return hits / k

def recall_at_k(recommended, relevant, k=5):
    recommended_top_k = recommended[:k]
    hits = sum([1 for item in recommended_top_k if item in relevant])
    return hits / len(relevant) if relevant else 0

In [20]:
student_profile = """
Ik heb een achtergrond in de zorg en ik ben geïnteresseerd in hoe technologie kan helpen om zorg en welzijn slimmer te organiseren. 
Dingen als smart health, e-health, zorg op afstand en innovatieve oplossingen voor ouderen spreken me erg aan. 
Ik werk graag met praktische toepassingen van technologie in de zorgpraktijk en vind het leuk om in teams aan echte cases te werken.
"""

relevant_modules = [
    176, 171, 172, 173
]

def evaluate_recommendations(predicted_ids, relevant_ids):
    results = []
    for module_id in predicted_ids:
        hit = module_id in relevant_ids
        results.append({
            "id": module_id,
            "name": df.loc[df["id"] == module_id, "name"].values[0],
            "status": "GOED" if hit else "FOUT"
        })
    return pd.DataFrame(results)


result = recommend_modules(student_profile)
predicted_ids = list(result["id"])

precision = precision_at_k(predicted_ids, relevant_modules, k=5)
recall = recall_at_k(predicted_ids, relevant_modules, k=5)

display(evaluate_recommendations(predicted_ids, relevant_modules))
precision, recall

,id,name,status
0,176,Technologie die ècht werkt: innovatie in zorg en welzijn,GOED
1,171,Technologie in zorg en welzijn,GOED
2,172,Zorg dichtbij,GOED
3,173,Langer thuis in de wijk,GOED
4,199,Acuut Complexe Zorg,FOUT


(0.8, 1.0)

In [21]:
edge_cases = {
    "leeg": "",
    "stopwoorden": "de en het maar want",
    "heel kort": "zorg",
    "heel kort 2": "technologie",
    "heel lang": " ".join(["zorg technologie innovatie welzijn"] * 50),
    "irrelevant": "voetbal gamen minecraft hond koekjes bakken",
    "engels": "I am interested in smart health and technology in healthcare",
    "extreem breed": "zorg technologie data gezondheid business energie onderwijs creativiteit duurzaamheid",
    "extreem specifiek": "Ik wil VR toepassen in de psychiatrie met realtime EEG-datastreams en biofeedback sensoren"
}

for label, profile in edge_cases.items():
    print(f"\n=== Edge case: {label} ===")
    
    try:
        res = recommend_modules(profile, top_n=5)
        display(res[["id", "name", "similarity", "similarity_raw", "match_terms", "reason"]])
    except Exception as e:
        print("Error:", e)


=== Edge case: leeg ===


,id,name,similarity,similarity_raw,match_terms,reason
0,159,Kennismaking met Psychologie,0.0,0.0,[],Deze module lijkt inhoudelijk redelijk aan te sluiten bij wat je interessant vindt.
1,160,Learning and working abroad,0.0,0.0,[],Deze module lijkt inhoudelijk redelijk aan te sluiten bij wat je interessant vindt.
2,161,Proactieve zorgplanning,0.0,0.0,[],Deze module sluit redelijk aan bij je interesses op basis van tekstuele overeenkomsten.
3,162,Rouw en verlies,0.0,0.0,[],Deze module sluit redelijk aan bij je interesses op basis van tekstuele overeenkomsten.
4,163,Acuut complexe zorg,0.0,0.0,[],Deze module sluit redelijk aan bij je interesses op basis van tekstuele overeenkomsten.



=== Edge case: stopwoorden ===


,id,name,similarity,similarity_raw,match_terms,reason
0,159,Kennismaking met Psychologie,0.0,0.0,[],Op basis van de overeenkomst tussen jouw profiel en de modulebeschrijving lijkt deze module redelijk bij je te passen.
1,160,Learning and working abroad,0.0,0.0,[],Op basis van de overeenkomst tussen jouw profiel en de modulebeschrijving lijkt deze module redelijk bij je te passen.
2,161,Proactieve zorgplanning,0.0,0.0,[],Op basis van de overeenkomst tussen jouw profiel en de modulebeschrijving lijkt deze module redelijk bij je te passen.
3,162,Rouw en verlies,0.0,0.0,[],Deze module sluit redelijk aan bij je interesses op basis van tekstuele overeenkomsten.
4,163,Acuut complexe zorg,0.0,0.0,[],Op basis van de overeenkomst tussen jouw profiel en de modulebeschrijving lijkt deze module redelijk bij je te passen.



=== Edge case: heel kort ===


...

,id,name,similarity,similarity_raw,match_terms,reason
0,320,Smart Health,1.000000,0.805223,"[health, smart, smart health]","Je interesse in 'health', 'smart' en 'smart health' komt duidelijk terug in 'Smart Health', waardoor deze module erg goed bij je aansluit."
1,319,Green Belt,0.789084,0.635388,"[health, smart, smart health]","In 'Green Belt' komen 'health', 'smart' en 'smart health' aan bod, wat goed aansluit bij jouw interesses."
2,324,Engineering for non-engineers,0.692796,0.557856,"[health, smart, smart health]","Omdat 'health', 'smart' en 'smart health' centraal staan in 'Engineering for non-engineers', past deze module goed bij jouw interesses."
3,318,AI Driven Robotics,0.682996,0.549964,"[health, smart, smart health]","Je interesse in 'health', 'smart' en 'smart health' komt duidelijk terug in 'AI Driven Robotics', waardoor deze module goed bij je aansluit."
4,176,Technologie die ècht werkt: innovatie in zorg en welzijn,0.434709,0.350038,"[health, smart, smart health]","In 'Technologie die ècht werkt: innovatie in zorg en welzijn' komen 'health', 'smart' en 'smart health' aan bod, wat goed aansluit bij jouw interesses."



=== Edge case: extreem breed ===


,id,name,similarity,similarity_raw,match_terms,reason
0,172,Zorg dichtbij,1.000000,0.344179,"[technologie, zorg]","In 'Zorg dichtbij' komen 'technologie' en 'zorg' aan bod, wat goed aansluit bij jouw interesses."
1,171,Technologie in zorg en welzijn,0.880104,0.302913,"[technologie, zorg]","Je interesse in 'technologie' en 'zorg' komt duidelijk terug in 'Technologie in zorg en welzijn', waardoor deze module erg goed bij je aansluit."
2,259,Module 3.4 Duurzaam gebouwde omgeving - Gezondheid en zorg,0.797525,0.274491,"[gezondheid, zorg]","In 'Module 3.4 Duurzaam gebouwde omgeving - Gezondheid en zorg' komen 'gezondheid' en 'zorg' aan bod, wat goed aansluit bij jouw interesses."
3,176,Technologie die ècht werkt: innovatie in zorg en welzijn,0.783191,0.269557,"[technologie, zorg]","Omdat 'technologie' en 'zorg' centraal staan in 'Technologie die ècht werkt: innovatie in zorg en welzijn', past deze module goed bij jouw interesses."
4,316,Business Innovation,0.676187,0.232729,[creativiteit],"In 'Business Innovation' komen 'creativiteit' aan bod, wat goed aansluit bij jouw interesses."



=== Edge case: extreem specifiek ===


,id,name,similarity,similarity_raw,match_terms,reason
0,159,Kennismaking met Psychologie,0.0,0.0,[],Deze module sluit redelijk aan bij je interesses op basis van tekstuele overeenkomsten.
1,160,Learning and working abroad,0.0,0.0,[],Op basis van de overeenkomst tussen jouw profiel en de modulebeschrijving lijkt deze module redelijk bij je te passen.
2,161,Proactieve zorgplanning,0.0,0.0,[],Op basis van de overeenkomst tussen jouw profiel en de modulebeschrijving lijkt deze module redelijk bij je te passen.
3,162,Rouw en verlies,0.0,0.0,[],Op basis van de overeenkomst tussen jouw profiel en de modulebeschrijving lijkt deze module redelijk bij je te passen.
4,163,Acuut complexe zorg,0.0,0.0,[],Deze module lijkt inhoudelijk redelijk aan te sluiten bij wat je interessant vindt.
